## Integrantes
- Nelson García  
- Diego Linares
- Joaquin Puente
- José Mérida
- Joaquín Campos

# Análisis exploratorio

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from ydata_profiling import ProfileReport

: 

In [ ]:
# estaremos definiendo ambos csvs para poder tener acceso a ambos pero usaremos el train y luego sobre el test replicaremos una vez se considere importante
dftest = pd.read_csv("../data/test.csv")

# Definir NA como nuestros NaN
dftrain = pd.read_csv("../data/train.csv", na_values=["NA"])

# incluimos en el analissi exploratorio lo basico para poder tenerlo a la mano
dftrain.head()

In [ ]:
# datos estadísticos básicos 
dftrain.describe()

In [ ]:
# tipos
dftrain.info()

### Análisis Exploratorio


## ¿Qué variables podriamos eliminar?  

In [ ]:
# Revisamos posibles variables redundantes entre sí mismas
print(dftrain.nunique())

### Evaluaciones entre variables 2darias para ver redundancia con la 1


In [ ]:
# condition
condition_diff_count = (dftrain["Condition1"] != dftrain["Condition2"]).sum()
print(f"Number of rows with different Condition1 and Condition2: {condition_diff_count}")
# exterior
exterior_diff_count = (dftrain["Exterior1st"] != dftrain["Exterior2nd"]).sum()
print(f"Number of rows with different Exterior1st and Exterior2nd: {exterior_diff_count}")
# BsmtFinSF2
bsmt_diff_count = (dftrain["BsmtFinSF1"] != dftrain["BsmtFinSF2"]).sum()
print(f"Number of rows with different BsmtFinSF1 and BsmtFinSF2: {bsmt_diff_count}")



En conclusion podemos definir que las variables 2darias no las podemos eliminar del todo porque numeros como 195 son pesados o voluminosos para posibles calculos en el futuro con SalesPrice


### Variables con valores nulos


In [ ]:
# Verificar cuántos valores nulos hay en cada columna
missing_values = dftrain.isnull().sum() / len(dftrain) * 100
print(missing_values.sort_values(ascending=False))

Con este output podemos eliminar 4 variables irrelevantes porque tienen NaN en 80% o más (recordemos que NA se cambio por el NaN al leer en el csv)

## ¿El Id valdrá la pena?
Al ser el Id solo un identificador unico de las casas no es relevante para el valor que estamos buscando predecir

In [ ]:
dftrain_original = dftrain.copy()  # Guarda una copia por si acaso
dftrain = dftrain.drop(["Id", "MiscFeature", "PoolQC", "Alley", "Fence"], axis=1)

: 

In [ ]:
#Para verificar que se han eliminado.
print(dftrain.columns.tolist())

Despues de eliminar esas columnas, podriamos volver a contar valores nulos por columna. 

In [ ]:
# Contar valores nulos por columna
missing_values = dftrain.isnull().sum().sort_values(ascending=False)
missing_values[missing_values > 0]

Para variables nulos en numericas se reemplazaran con la mediana.

In [ ]:
# Reemplazo en numéricas
num_cols = dftrain.select_dtypes(include=["number"]).columns
dftrain[num_cols] = dftrain[num_cols].fillna(dftrain[num_cols].median())